# SegFast: comparison and examples

In [1]:
import numpy as np
import pandas as pd

import segyio
import segfast

from batchflow import Notifier
from batchflow.plotter import plot

In [2]:
path_sgy = 'cube.sgy'
path_horizon = 'horizon.char'

## Basic operations (load headers and traces/lines)

In [3]:
segfast_file = segfast.open(path_sgy)

In [4]:
segfast_file.load_traces([123, 333, 777], buffer=None)

array([[   0.   ,    0.   ,    0.   , ...,    0.   ,    0.   ,    0.   ],
       [   0.   ,    0.   ,    0.   , ...,    0.   ,    0.   ,    0.   ],
       [   0.   ,    0.   ,    0.   , ..., 1364.876, 1023.657,  341.219]],
      dtype=float32)

In [5]:
segfast_file.load_depth_slices([5, 10, 15], buffer=None)

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [6]:
segfast_file.load_headers(['INLINE_3D', 'CROSSLINE_3D'])

,TRACE_SEQUENCE_FILE,INLINE_3D,CROSSLINE_3D
0,1,24,19
1,2,24,20
2,3,24,21
3,4,24,22
4,5,24,23
...,...,...,...
3611262,3611263,2586,1423
3611263,3611264,2586,1424
3611264,3611265,2586,1425
3611265,3611266,2586,1426


## Comparison with `segyio`

In [7]:
HEADERS = ['INLINE_3D', 'CROSSLINE_3D']
LINE_IDX = 500
DEPTH_IDX = 100

segyio_file = segyio.open(path_sgy, strict=True, ignore_geometry=False)
segfast_file = segfast.open(path_sgy)

### Headers

In [8]:
%%time

headers_segyio = pd.DataFrame({header : segyio_file.attributes(getattr(segyio.TraceField, header))
                               for header in HEADERS})

CPU times: user 29.9 s, sys: 7.48 s, total: 37.4 s
Wall time: 37.1 s


In [9]:
%%timeit

headers_segfast = segfast_file.load_headers(HEADERS)

562 ms ± 7.98 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Inline

In [10]:
%%timeit
slide_segyio_i = segyio_file.iline[segyio_file.ilines[LINE_IDX]]

2.74 ms ± 160 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [11]:
headers_segfast = segfast_file.load_headers(HEADERS)
n_ilines = len(np.unique(headers_segfast['INLINE_3D']))
n_xlines = len(np.unique(headers_segfast['CROSSLINE_3D']))

In [12]:
%%timeit

start = LINE_IDX * n_xlines
stop = start + n_xlines
indices = range(start, stop)

slide_segfast_i = segfast_file.load_traces(indices=indices)

1.55 ms ± 5.63 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


Here we proposed that traces of iline are stored sequentially. In the common situation you have to use information from headers. So you can compute auxiliary matrix once and then use it to load lines.

In [13]:
headers = segfast_file.load_headers(HEADERS)
traces_idx = headers.reset_index().pivot(index='INLINE_3D', columns='CROSSLINE_3D', values='index').values

In [14]:
%%timeit

slide_segfast_i_from_header = segfast_file.load_traces(indices=traces_idx[LINE_IDX])

1.48 ms ± 4.85 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Crossline

In [15]:
%%timeit

slide_segyio_x = segyio_file.xline[segyio_file.xlines[LINE_IDX]]

8.94 ms ± 60.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [16]:
%%timeit

slide_segfast_x_from_header = segfast_file.load_traces(indices=traces_idx[:, LINE_IDX])

3.78 ms ± 49 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### Depth

In [17]:
%%timeit
slide_segyio_d = segyio_file.depth_slice[DEPTH_IDX]

3.94 s ± 5.03 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [18]:
%%timeit
slide_segfast_d = segfast_file.load_depth_slices([DEPTH_IDX]).reshape(n_ilines, n_xlines)

73.4 ms ± 703 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Example: `Geometry`/`Field` classes

In [19]:
class Geometry:
    INDEX_HEADERS_POSTSTACK = ('INLINE_3D', 'CROSSLINE_3D')

    def __init__(self, path, engine='memmap'):
        self.path = path
        self.file = segfast.open(path, engine=engine)
        self._headers = None
        
        self.compute_stats()

    def compute_stats(self):
        self.ilines = np.unique(self.headers['INLINE_3D'])
        self.xlines = np.unique(self.headers['CROSSLINE_3D'])
        self.n_ilines, self.n_xlines = len(self.ilines), len(self.xlines)
        self.shape = (self.n_ilines, self.n_xlines, self.file.n_samples)
        self.shifts = min(self.ilines), min(self.xlines)
        self.delay = self.file.delay
        self.traces_idx = self.headers.reset_index().pivot(index='INLINE_3D', columns='CROSSLINE_3D', values='index').values

    @property
    def headers(self):
        if self._headers is None:
            self._headers = self.file.load_headers(self.INDEX_HEADERS_POSTSTACK)
        return self._headers

    def load_slide(self, loc, axis=0):
        name = 'INLINE_3D' if axis == 0 else 'CROSSLINE_3D'
        return self.file.load_traces(indices=self.headers[headers_segfast[name] == idx]['TRACE_SEQUENCE_FILE'])

    def load_depth(self, loc):
        return self.file.load_depth_slices([loc]).reshape(self.n_ilines, self.n_xlines)

    def load_value(self, index, limits):
        return self.file.load_traces([index], limits=limits)[0]

In [20]:
from numba import njit, prange

def get_points(data, points, buffer, traces_idx, shift):
    for i in prange(len(points)):
        item = points[i]
        buffer[item[0], item[1]] = data[
            item[2] - shift[0]:item[2] - shift[1],
            traces_idx[item[0], item[1]]
        ]
    return buffer

class Horizon:
    def __init__(self, path, geometry):
        self.path = path
        self.geometry = geometry

        self.load()

    def load(self):
        self.absolute_points = pd.read_csv(self.path, header=None, sep='\s+', names=['INLINE_3D', 'CROSSLINE_3D', 'DEPTH']).values.astype('int32')
        self.points = self.absolute_points - np.array([*self.geometry.shifts, self.geometry.delay])
        self.points[:, 2] = self.points[:, 2] / (self.geometry.file.sample_interval // 1000)
        self.matrix = np.full(self.geometry.shape[:2], fill_value=np.nan)
        self.matrix[self.points[:, 0], self.points[:, 1]] = self.points[:, 2]
        self.d_min, self.d_max = int(np.nanmin(self.matrix)), int(np.nanmax(self.matrix))
        
    def show(self):
        plot(self.matrix, colorbar=True, cmap='viridis')

    def get_cube_values(self, window=1, bar=True, numba=False):
        data = self.geometry.file.load_depth_slices(range(self.d_min - window // 2, self.d_max + (window - window // 2)))
        buffer = np.full((*self.geometry.shape[:2], window), fill_value=np.nan)

        if numba:
            parallel = numba == 'parallel'
            get_points_ = njit(parallel=parallel)(get_points)
        else:
            get_points_ = get_points

        return get_points_(data, self.points, buffer, self.geometry.traces_idx, (self.d_min, self.d_min - window))


    def get_cube_values_fast(self, window=1, bar=True, numba=False, max_workers=40):
        min_depth, max_depth = self.d_min - window // 2, self.d_max + (window - window // 2) + 1
        data = np.empty((max_depth - min_depth + window, self.geometry.shape[0] * self.geometry.shape[1]))#, fill_value=np.nan)

        with ThreadPoolExecutor(max_workers=max_workers) as executor:

            def func(start, stop):
                self.geometry.file.load_depth_slices(range(start, stop), buffer=data[start - min_depth:stop - min_depth])

            step = (max_depth - min_depth) // max_workers
            for start in range(min_depth, max_depth, step):
                stop = min(start + step, max_depth)
                future = executor.submit(func, start=start, stop=stop)

        buffer = np.full((*self.geometry.shape[:2], window), fill_value=np.nan)

        if numba:
            parallel = numba == 'parallel'
            get_points_ = njit(parallel=parallel)(get_points)
        else:
            get_points_ = get_points

        return get_points_(data, self.points, buffer, self.geometry.traces_idx, (self.d_min, self.d_min - window))

Let's create instances of geometry and horizon

In [21]:
geometry = Geometry(path_sgy, engine='memmap')
horizon = Horizon(path_horizon, geometry=geometry)

Now we can load amplitudes along horizon in some window (10 ticks above and 10 ticks below)

In [22]:
%%time

window = 20
values = horizon.get_cube_values(window)

CPU times: user 16.6 s, sys: 2.09 s, total: 18.7 s
Wall time: 18.7 s


Let's check how long is loading of data

In [23]:
%%time

print(range(horizon.d_min - window // 2, horizon.d_max + (window - window // 2)))
geometry.file.load_depth_slices(range(horizon.d_min - window // 2, horizon.d_max + (window - window // 2)))

range(1050, 1224)
CPU times: user 12.9 s, sys: 1.18 s, total: 14.1 s
Wall time: 14 s


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

## Speed up loading

In [24]:
from concurrent.futures import ThreadPoolExecutor
from functools import partial

min_depth, max_depth = horizon.d_min - window // 2, horizon.d_max + (window - window // 2) + 1
buffer = np.empty((max_depth - min_depth + window, geometry.shape[0] * geometry.shape[1]))

Loading can be faster with parallel threads

In [25]:
%%time

max_workers = 40
with ThreadPoolExecutor(max_workers=max_workers) as executor:

    def func(start, stop):
        buffer[start - min_depth:stop - min_depth] = geometry.file.load_depth_slices(range(start, stop))

    step = (max_depth - min_depth) // max_workers
    for start in range(min_depth, max_depth, step):
        stop = min(start + step, max_depth)
        future = executor.submit(func, start=start, stop=stop)

CPU times: user 22.3 s, sys: 10.2 s, total: 32.4 s
Wall time: 1.87 s


Here we can get values along horizon with such parallelization

In [26]:
%%time

values = horizon.get_cube_values_fast(window, numba=True, max_workers=max_workers)

CPU times: user 22.7 s, sys: 6.46 s, total: 29.2 s
Wall time: 1.81 s
